In [ ]:
import numpy as np
import tensorflow as tf

import os
import json
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import cv2
from utils_train.Datagenerator import Dataset_COCO, Dataset_Pascal, Dataset_COCO_Temp

In [ ]:
modelName = "MobileNetV3_FPN_TTFNet"

model_dir = "checkpoints/"
modelPart = modelName.split("_")

with open(os.path.join("model", "0_Config", modelName+".json"), "r") as config_file:
    config = json.load(config_file)

config['modelName'] = modelName
config['training_config']['num_classes'] = 80
config['training_config']['batch_size'] = 1

train_dataset = Dataset_COCO(config, mode = 'train')

for data in train_dataset.dataset:
    break

batch_idx = 0
image = data[0][batch_idx, ...]
hm  = data[1][batch_idx][..., :80]
box  = data[1][batch_idx][..., 80:84]
rW  = data[1][batch_idx][..., 84:85]

In [ ]:
show_img = (image.numpy()+1)/2
colors = np.random.rand(80, 3)*255

bboxes = tf.reshape(box, [-1, 4])
bboxes = tf.boolean_mask(bboxes, tf.reduce_any(bboxes > 0, axis=1))

h,w, _ = tf.unstack(tf.cast(tf.shape(show_img), tf.float32))

for bbox in bboxes:
    y1, x1, y2, x2 = bbox
    x1 = int(x1*w)
    x2 = int(x2*w)
    y1 = int(y1*h)
    y2 = int(y2*h)
    cv2.rectangle(show_img, (x1, y1), (x2, y2), colors[1], 1)

plt.figure(figsize=(12,12))
plt.imshow(show_img)

plt.figure(figsize=(12,12))
plt.imshow(tf.reduce_max(hm, -1).numpy(), cmap='viridis')

plt.figure(figsize=(12,12))
box_np = box.numpy()
plt.imshow(box_np)

plt.figure(figsize=(12,12))
rW_show = rW.numpy()
plt.imshow(rW_show)

a1 = tf.cast(hm==1, tf.float32)
print("Num of Positives: ", tf.reduce_sum(a1))

In [ ]:
print("TP1: ", tf.reduce_max(box_np).numpy(), tf.reduce_min(box_np).numpy())
print("TP2: ", tf.reduce_max(hm).numpy(), tf.reduce_min(hm).numpy())